In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_380.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_517.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_364.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_258.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_237.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_476.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_563.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_159.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_464.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_518.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_141.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_398.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_442.jpg
/kaggle/input/cats-and-dogs-image-classification/test/dogs/dog_443.jpg
/kaggl

In [2]:
!pip install torch torchvision 

In [3]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets

In [4]:
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 128 * 128, 256)  # Adjust the input size based on your image dimensions
        self.fc2 = nn.Linear(256, 279)  # Adjust num_classes based on your dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, current_loss, model):
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True  # Stop training
        return False
    
# Set up the data transformations and create the dataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

data_path = "/kaggle/input/cats-and-dogs-image-classification/train"
batch_size = 64
num_classes = len(os.listdir(data_path))  # Assuming each subfolder is a class

train_dataset = ImageFolder(root=data_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set up learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=8, verbose=True)

# Set up early stopping
early_stopping = EarlyStopping(patience=32, verbose=True)

# Training loop
# num_epochs = 100
num_epochs = 30  # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average loss for the epoch
    average_loss = total_loss / len(train_loader)

    # Update learning rate based on validation loss
    validation_loss =0.0000001  # Add your validation loss calculation here
    lr_scheduler.step(validation_loss)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

    # Check for early stopping
    if early_stopping(validation_loss, model):
        print("Early stopping triggered.")
        break

# Save the trained model if needed
torch.save(model.state_dict(), "trained_model_with_dropout_and_early_stopping.pth")

Epoch 1/30, Average Loss: 25.9145
Epoch 2/30, Average Loss: 10.0281
Epoch 3/30, Average Loss: 5.4773
Epoch 4/30, Average Loss: 2.6515
Epoch 5/30, Average Loss: 1.3913
Epoch 6/30, Average Loss: 0.9131
Epoch 7/30, Average Loss: 0.5753
Epoch 8/30, Average Loss: 0.4725
Epoch 9/30, Average Loss: 0.3238
Epoch 00010: reducing learning rate of group 0 to 3.0000e-04.
Epoch 10/30, Average Loss: 0.2167
Epoch 11/30, Average Loss: 0.1545
Epoch 12/30, Average Loss: 0.1363
Epoch 13/30, Average Loss: 0.1227
Epoch 14/30, Average Loss: 0.1085
Epoch 15/30, Average Loss: 0.0954
Epoch 16/30, Average Loss: 0.0856
Epoch 17/30, Average Loss: 0.0783
Epoch 18/30, Average Loss: 0.0682
Epoch 00019: reducing learning rate of group 0 to 9.0000e-05.
Epoch 19/30, Average Loss: 0.0607
Epoch 20/30, Average Loss: 0.0559
Epoch 21/30, Average Loss: 0.0534
Epoch 22/30, Average Loss: 0.0519
Epoch 23/30, Average Loss: 0.0499
Epoch 24/30, Average Loss: 0.0492
Epoch 25/30, Average Loss: 0.0479
Epoch 26/30, Average Loss: 0.0467

In [5]:
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 128 * 128, 256)  # Adjust the input size based on your image dimensions
        self.fc2 = nn.Linear(256, 279)  # Adjust num_classes based on your dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set up the data transformations and create the test dataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

test_data_path = "/kaggle/input/cats-and-dogs-image-classification/test"
batch_size = 32

test_dataset = ImageFolder(root=test_data_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing

# Initialize the model and load the trained weights
model = SimpleCNN()
model.load_state_dict(torch.load("/kaggle/working/trained_model_with_dropout_and_early_stopping.pth"))  # Load the weights saved during training

# Set the model to evaluation mode
model.eval()

# Define a function to evaluate the model on the test set
def evaluate_model(model, test_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    total_correct = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    average_loss = total_loss / len(test_loader)

    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    print(f"Average Test Loss: {average_loss:.4f}")

# Use the evaluation function
criterion = nn.CrossEntropyLoss()
evaluate_model(model, test_loader, criterion)

Test Accuracy: 59.29%
Average Test Loss: 0.9687
